In [1]:
#!/usr/bin/env python
# encoding: utf-8

"""
@Author: yangwenhao
@Contact: 874681044@qq.com
@Software: PyCharm
@File: cam_2.py
@Time: 2021/4/12 21:47
@Overview:
    Created on 2019/8/4 上午9:37
    @author: mick.yi
"""

import os
import pdb

import numpy as np
import torch
from torch.nn.parallel.distributed import DistributedDataParallel

from Define_Model.ResNet import ThinResNet

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"
torch.distributed.init_process_group(backend="nccl", init_method='tcp://localhost:12556', rank=0,
                                     world_size=1)

In [3]:
class GradCAM(object):
    """
    1: 网络不更新梯度,输入需要梯度更新
    2: 使用目标类别的得分做反向传播
    """

    def __init__(self, net, layer_name):
        self.net = net
        self.layer_name = layer_name
        self.feature = {}
        self.gradient = {}
        self.net.eval()
        self.handlers = []
        self._register_hook()

    def _get_features_hook(self, module, input, output):
        print(type(module))
        if isinstance(self.net, DistributedDataParallel):
            self.feature[input[0].device] = output[0]
        else:
            self.feature = output[0]
#         print("Device {}, forward out feature shape:{}".format(input[0].device, output[0].size()))

    def _get_grads_hook(self, module, input_grad, output_grad):
        """
        :param input_grad: tuple, input_grad[0]: None
                                   input_grad[1]: weight
                                   input_grad[2]: bias
        :param output_grad:tuple,长度为1
        :return:
        """
        if isinstance(self.net, DistributedDataParallel):
            if input_grad[0].device not in self.gradient:
                self.gradient[input_grad[0].device] = output_grad[0]
            else:
                self.gradient[input_grad[0].device] += output_grad[0]
        else:
            self.gradient += output_grad[0]
        
#         print(output_grad[0])
#         print("Device {}, backward out gradient shape:{}".format(input_grad[0].device, output_grad[0].size()))

    def _register_hook(self):

        if isinstance(self.net, DistributedDataParallel):
            modules = self.net.module.named_modules()
        else:
            modules = self.net.named_modules()

        for (name, module) in modules:
            if name == self.layer_name:
                self.handlers.append(module.register_backward_hook(self._get_features_hook))
                self.handlers.append(module.register_backward_hook(self._get_grads_hook))

    def remove_handlers(self):
        for handle in self.handlers:
            handle.remove()

    def __call__(self, inputs, index):
        """
        :param inputs: [1,3,H,W]
        :param index: class id
        :return:
        """
#         self.net.zero_grad()

        output, _ = self.net(inputs)  # [1,num_classes]
        pdb.set_trace()

        if index is None:
            index = torch.argmax(output)
            
        target = output.gather(1, index)# .mean()

        # target = output[0][index]
        for i in target:
            i.backward(retain_graph=True)
        
        if isinstance(self.net, DistributedDataParallel):
            feature = []
            gradient = []
            for d in self.gradient:
                feature.append(self.feature[d])
                gradient.append(self.gradient[d])

            feature = torch.cat(feature, dim=0)
            gradient = torch.cat(gradient, dim=0)
        else:
            feature = self.feature
            gradient = self.gradient
            
        return feature, gradient
        
        # gradient = self.gradient[0].cpu().data.numpy()  # [C,H,W]
        # weight = np.mean(gradient, axis=(1, 2))  # [C]
        # feature = self.feature[0].cpu().data.numpy()  # [C,H,W]

        # cam = feature * weight[:, np.newaxis, np.newaxis]  # [C,H,W]
        # cam = np.sum(cam, axis=0)  # [H,W]
        # cam = np.maximum(cam, 0)  # ReLU
        #
        # # 数值归一化
        # cam -= np.min(cam)
        # cam /= np.max(cam)
        # # resize to 224*224
        # cam = cv2.resize(cam, (224, 224))
        # return cam
        
#         print("gradient shape: ", gradient.shape)
#         print("feature shape: ", feature.shape)

class Sum_GradCAM(object):
    """
    1: 网络不更新梯度,输入需要梯度更新
    2: 使用目标类别的得分做反向传播
    """

    def __init__(self, net, layer_name):
        self.net = net
        self.layer_name = layer_name
        self.feature = {}
        self.gradient = {}
        self.net.eval()
        self.handlers = []
        self._register_hook()

    def _get_features_hook(self, module, input, output):
        
        if isinstance(self.net, DistributedDataParallel):
            self.feature[input[0].device] = output[0]
        else:
            self.feature = output[0]
#         print("Device {}, forward out feature shape:{}".format(input[0].device, output[0].size()))

    def _get_grads_hook(self, module, input_grad, output_grad):
        """
        :param input_grad: tuple, input_grad[0]: None
                                   input_grad[1]: weight
                                   input_grad[2]: bias
        :param output_grad:tuple,长度为1
        :return:
        """
        if isinstance(self.net, DistributedDataParallel):
            if input_grad[0].device not in self.gradient:
                self.gradient[input_grad[0].device] = output_grad[0]
            else:
                self.gradient[input_grad[0].device] += output_grad[0]
        else:
            self.gradient = output_grad[0]
        
#         print(output_grad[0])
#         print("Device {}, backward out gradient shape:{}".format(input_grad[0].device, output_grad[0].size()))

    def _register_hook(self):

        if isinstance(self.net, DistributedDataParallel):
            modules = self.net.module.named_modules()
        else:
            modules = self.net.named_modules()

        for (name, module) in modules:
            if name == self.layer_name:
                self.handlers.append(module.register_backward_hook(self._get_features_hook))
                self.handlers.append(module.register_backward_hook(self._get_grads_hook))

    def remove_handlers(self):
        for handle in self.handlers:
            handle.remove()

    def __call__(self, inputs, index):
        """
        :param inputs: [1,3,H,W]
        :param index: class id
        :return:
        """
#         self.net.zero_grad()

        output, _ = self.net(inputs)  # [1,num_classes]
        pdb.set_trace()

        if index is None:
            index = torch.argmax(output)
            
        target = output.gather(1, index).mean()
        target.backward(retain_graph=True)
        
        if isinstance(self.net, DistributedDataParallel):
            feature = []
            gradient = []
            for d in self.gradient:
                feature.append(self.feature[d])
                gradient.append(self.gradient[d])

            feature = torch.cat(feature, dim=0)
            gradient = torch.cat(gradient, dim=0)
        else:
            feature = self.feature
            gradient = self.gradient
        
        return feature, gradient
        
#         print("gradient shape: ", gradient.shape)
#         print("feature shape: ", feature.shape)

In [4]:
model = ThinResNet()
model = model.cuda()
model = DistributedDataParallel(model)
gc = GradCAM(model, 'layer4')

/home/yangwenhao/local/project/SpeakerVerification-pytorch/Define_Model/ResNet.py:374: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(m.weight, mean=0., std=1.)
/home/yangwenhao/local/project/SpeakerVerification-pytorch/Define_Model/ResNet.py:376: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.weight, 1)
/home/yangwenhao/local/project/SpeakerVerification-pytorch/Define_Model/ResNet.py:377: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.bias, 0)


In [5]:
x = torch.randn((20, 1, 224, 224)).cuda() # *1.2 +1.
l = torch.range(0, 19).long().unsqueeze(1).cuda()
y = model(x)

#
cam = gc(x, l)
# print(cam.shape)

/home/yangwenhao/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  


> <ipython-input-3-194ac11b6417>(70)__call__()
-> if index is None:
(Pdb) c
<class 'torch.nn.modules.container.Sequential'><class 'torch.nn.modules.container.Sequential'>

<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torch.nn.modules.container.Sequential'>
<class 'torc